<a href="https://colab.research.google.com/github/AleAguirreM/Analitica3AA/blob/main/d_modelos2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib

In [3]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162985 sha256=f41d1c781fe3c0b23fede4331badd13642ce4ed3b11d4dbd844e1b37a6a09470
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [4]:
####Paquete para sistemas de recomendación surprise

from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split




In [5]:
#############################################
#### conectar_base_de_Datos#################
############################################

conn = sql.connect("/content/drive/MyDrive/db_moviesSQL")
curr=conn.cursor()
curr.execute("SELECT name FROM sqlite_master WHERE type = 'table'")
curr.fetchall()

[('ratings',),
 ('movies',),
 ('reco',),
 ('movies_sel',),
 ('movies_n',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',),
 ('full',),
 ('x',)]

In [6]:
#######################################################################
#### 3 Sistema de recomendación basado en contenido KNN #################
#### Con base en todo lo visto por el usuario #######################
#######################################################################

movies =pd.read_sql('select * from full', conn )
movies ['year']=movies.year.astype('int')

In [7]:
##### cargar data frame escalado y con dummies ###

movies_dum1= joblib.load('/content/drive/MyDrive/salidas_movies_dum1.joblib')
dumm2= joblib.load('/content/drive/MyDrive/salidas_dumm2.joblib')

In [8]:
#### seleccionar usuario para recomendaciones ####

usuarios=pd.read_sql('select distinct (userId) as user_id from ratings_final',conn)

user_id=1 ### para ejemplo manual

In [15]:
def recomendar(user_id=list(usuarios['user_id'].value_counts().index)):

    ###seleccionar solo los ratings del usuario seleccionado
    ratings=pd.read_sql('select * from full where userId=:user',conn, params={'user':user_id,})


    ###convertir ratings del usuario a array
    l_movies_r=ratings['movieId'].to_numpy()

    ###agregar la columna de isbn y titulo del libro a dummie para filtrar y mostrar nombre
    dumm2[['movieId','title']]=movies[['movieId','title']]

    ### filtrar libros calificados por el usuario
    movies_r= dumm2[dumm2['movieId'].isin(l_movies_r)]

    ## eliminar columna nombre e isbn
    movies_r=movies_r.drop(columns=['movieId','title'])
    movies_r["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
    ##centroide o perfil del usuario
    centroide=movies_r.groupby("indice").mean()


    ### filtrar libros no leídos
    movies_nr=dumm2[~dumm2['movieId'].isin(l_movies_r)]
    ## eliminbar nombre e isbn
    movies_nr=movies_nr.drop(columns=['movieId','title'])

    ### entrenar modelo
    model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
    model.fit(movies_nr)
    dist, idlist = model.kneighbors(centroide)

    ids=idlist[0] ### queda en un array anidado, para sacarlo
    recomend_b=movies.loc[ids][['title','movieId']]
    leidos=movies[movies['movieId'].isin(l_movies_r)][['title','movieId']]

    return recomend_b


recomendar(2)
print(interact(recomendar))


interactive(children=(Dropdown(description='user_id', options=(1, 410, 403, 404, 405, 406, 407, 408, 409, 411,…

<function recomendar at 0x789e74db0ca0>


In [10]:
############################################################################
#####4 Sistema de recomendación filtro colaborativo #####
############################################################################

### datos originales en pandas
## knn solo sirve para calificaciones explicitas
ratings=pd.read_sql('select * from full where rating>0', conn)


####los datos deben ser leidos en un formato espacial para surprise
reader = Reader(rating_scale=(0, 10)) ### la escala de la calificación
###las columnas deben estar en orden estándar: user item rating
data   = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)


#####Existen varios modelos
models=[KNNBasic(),KNNWithMeans(),KNNWithZScore(),KNNBaseline()]
results = {}

###knnBasiscs: calcula el rating ponderando por distancia con usuario/Items
###KnnWith means: en la ponderación se resta la media del rating, y al final se suma la media general
####KnnwithZscores: estandariza el rating restando media y dividiendo por desviación
####Knnbaseline: calculan el desvío de cada calificación con respecto al promedio y con base en esos calculan la ponderación


#### for para probar varios modelos ##########
model=models[1]
for model in models:

    CV_scores = cross_validate(model, data, measures=["MAE","RMSE"], cv=5, n_jobs=-1)

    result = pd.DataFrame.from_dict(CV_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result


performance_df = pd.DataFrame.from_dict(results).T
performance_df.sort_values(by='RMSE')

,MAE,RMSE,fit_time,test_time
knns.KNNBaseline,0.639322,0.837289,0.275570,3.354260
knns.KNNWithZScore,0.643784,0.847108,0.251805,2.194962
knns.KNNWithMeans,0.648607,0.848579,0.279322,3.505415
knns.KNNBasic,0.687539,0.897910,0.294559,3.706307


In [11]:
###################se escoge el mejor knn withmeans#########################
param_grid = { 'sim_options' : {'name': ['msd','cosine'], \
                                'min_support': [5,2], \
                                'user_based': [False, True]}
             }

## min support es la cantidad de items o usuarios que necesita para calcular recomendación
## name medidas de distancia

### se afina si es basado en usuario o basado en ítem

gridsearchKNNWithMeans = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], \
                                      cv=2, n_jobs=-1)

gridsearchKNNWithMeans.fit(data)


gridsearchKNNWithMeans.best_params["rmse"]
gridsearchKNNWithMeans.best_score["rmse"]
gs_model=gridsearchKNNWithMeans.best_estimator['rmse'] ### mejor estimador de gridsearch


In [12]:

################# Entrenar con todos los datos y Realizar predicciones con el modelo afinado

trainset = data.build_full_trainset() ### esta función convierte todos los datos en entrnamiento, las funciones anteriores dividen  en entrenamiento y evaluación
model=gs_model.fit(trainset) ## se reentrena sobre todos los datos posibles (sin dividir)



predset = trainset.build_anti_testset() ### crea una tabla con todos los usuarios y las peliculas que han visto
#### en la columna de rating pone el promedio de todos los rating, en caso de que no pueda calcularlo para un item-usuario
len(predset)

predictions = gs_model.test(predset) ### función muy pesada, hace las predicciones de rating para todos los libros que no hay leido un usuario


####### la predicción se puede hacer para una pelicula
model.predict(uid=269397, iid='0446353205',r_ui='') ### uid debía estar en número e isb en comillas

predictions_df = pd.DataFrame(predictions) ### esta tabla se puede llevar a una base donde estarán todas las predicciones
predictions_df.shape
predictions_df.head()
predictions_df['r_ui'].unique() ### promedio de ratings
predictions_df.sort_values(by='est',ascending=False)

Computing the msd similarity matrix...
Done computing similarity matrix.


,uid,iid,r_ui,est,details
62904,53,1104,3.613952,5.789585,"{'actual_k': 14, 'was_impossible': False}"
62582,53,168252,3.613952,5.752156,"{'actual_k': 11, 'was_impossible': False}"
62962,53,1235,3.613952,5.646630,"{'actual_k': 13, 'was_impossible': False}"
62901,53,246,3.613952,5.636480,"{'actual_k': 12, 'was_impossible': False}"
61912,53,2959,3.613952,5.621156,"{'actual_k': 14, 'was_impossible': False}"
...,...,...,...,...,...
524426,442,1882,3.613952,0.000000,"{'actual_k': 19, 'was_impossible': False}"
181681,153,1499,3.613952,0.000000,"{'actual_k': 40, 'was_impossible': False}"
2902,3,63992,3.613952,0.000000,"{'actual_k': 15, 'was_impossible': False}"
2935,3,1882,3.613952,0.000000,"{'actual_k': 22, 'was_impossible': False}"


In [13]:
def recomendaciones(userId, n_recomend=10):
    predictions_userID = predictions_df[predictions_df['uid'] == userId].\
                    sort_values(by="est", ascending=False).head(n_recomend)

    recomendados = predictions_userID[['iid', 'est']]
    recomendados.to_sql('reco', conn, if_exists="replace")

    recomendados = pd.read_sql('''SELECT DISTINCT a.*, b.title
                                  FROM reco a
                                  LEFT JOIN full b
                                  ON a.iid = b.movieId''', conn)

    return recomendados

recomendaciones(userId=2, n_recomend=10)


,index,iid,est,title
0,1264,2959,4.319068,Fight Club (1999)
1,1513,750,4.334563,Dr. Strangelove or: How I Learned to Stop Worr...
2,1591,922,4.471084,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)
3,1691,858,4.322993,"Godfather, The (1972)"
4,1757,1267,4.409888,"Manchurian Candidate, The (1962)"
5,1758,1276,4.307865,Cool Hand Luke (1967)
6,2226,1204,4.339211,Lawrence of Arabia (1962)
7,2237,246,4.369698,Hoop Dreams (1994)
8,2240,1104,4.514940,"Streetcar Named Desire, A (1951)"
9,2299,1235,4.319128,Harold and Maude (1971)
